# Shapley Values - Game Theory

Reference: sage.game_theory.cooperative_game
http://doc.sagemath.org/html/en/reference/game_theory/sage/game_theory/cooperative_game.html

## Realization of a simple example

### Input: (Sagecell)

integer_function = {(): 0,
                    ('A',): 10,
                    ('B',): 10,
                    ('C',): 10,
                    ('A','B',): 30,
                    ('A','C',): 40,
                    ('B','C',): 40,
                    ('A','B','C',): 50}

integer_game = CooperativeGame(integer_function)

print(integer_game)

print(integer_game.shapley_value())

### Output: (Sagecell)

A 3 player co-operative game
{'A': 15, 'C': 20, 'B': 15}

Go to the website and paste the input above
https://sagecell.sagemath.org/


In [1]:
import pandas as pd
import numpy as np

#### Simulate the customer journey here  

In [11]:
# Assume a channel that makes no contribution to any coalition will get zero credit. 
customer_journey = {
    # Unordered Shapley Value Data
    'user0':{
        'Path': '',
        'Conversion': 0
    },
    'user1':{
        'Path': 'A',
        'Conversion': 10
    },
    'user2':{
        'Path': 'B',
        'Conversion': 10
    },
    'user3':{
        'Path': 'C',
        'Conversion': 10
    },
    'user4':{
        'Path': 'A>B',
        'Conversion': 30
    },
    'user5':{
        'Path': 'A>C',
        'Conversion': 40
    },
    'user6':{
        'Path': 'B>C',
        'Conversion': 40
    },
    'user7':{
        'Path': 'A>B>C',
        'Conversion': 50
    }

}
# Show the customer journey
customer_journey

{'user0': {'Path': '', 'Conversion': 0},
 'user1': {'Path': 'A', 'Conversion': 10},
 'user2': {'Path': 'B', 'Conversion': 10},
 'user3': {'Path': 'C', 'Conversion': 10},
 'user4': {'Path': 'A>B', 'Conversion': 30},
 'user5': {'Path': 'A>C', 'Conversion': 40},
 'user6': {'Path': 'B>C', 'Conversion': 40},
 'user7': {'Path': 'A>B>C', 'Conversion': 50}}

#### Re-construct the path and form a new dictionary

In [5]:
# Data Transformation
df = pd.DataFrame(customer_journey).transpose()
def split_fun(path):
    return path.split('>')

df['Path'] = df['Path'].apply(lambda x: tuple(split_fun(x)))

# Basic Shapley Values Stored in dictionary
base = dict(zip(df.Path, df.Conversion))

base

{('',): 0,
 ('A',): 10,
 ('B',): 10,
 ('C',): 10,
 ('A', 'B'): 30,
 ('A', 'C'): 40,
 ('B', 'C'): 40,
 ('A', 'B', 'C'): 50}

In [12]:
def binomial(n,k):
    return 1 if k==0 else (0 if n==0 else binomial(n-1, k) + binomial(n-1, k-1))

def powerset(x):
    if x == []:
        return [[]]
    
    s = powerset(x[1:])
    return s + [[x[0]] + i for i in s]

def v_utility(A, base):
    '''
    The utility function 𝜈(⋅) measures the contribution of a channel coalition
            - A : a coalition of channels.
            - base : A dictionnary containing the number of conversions that each subset of channels has yielded.
    '''
    s = powerset(A)
    mv = 0
    for x in s:
        x = tuple(x)
        if x in base:
            mv = base[x]
        
    return mv

In [13]:
# channels and utility function definition
v_values = {}
channels = ['A','B','C']
for A in powerset(channels):
    #  change List into tuple, then use it as keys.
    v_values[tuple(A)] = v_utility(A,base)

v_values

{(): 0,
 ('C',): 10,
 ('B',): 10,
 ('B', 'C'): 40,
 ('A',): 10,
 ('A', 'C'): 40,
 ('A', 'B'): 30,
 ('A', 'B', 'C'): 50}

\begin{align}
\phi_i(G) = \sum_{S \subseteq \Omega} \sum_{p \in S}
            \frac{1}{|S|\binom{N}{|S|}}
            \bigl( v(S) - v(S \setminus \{p\}) \bigr)
\end{align}

In [14]:
from collections import defaultdict

n = len(channels)
shapley_values = {}

for channel in channels:
    weighted_contribution = 0
    for coalition in powerset(channels):
        if coalition:
            k = len(coalition)
            t = tuple(p for p in coalition if p != channel)
            weight =  1 / (binomial(n, k)* k)
            weighted_contribution += weight * (v_values[tuple(coalition)]  - v_values[t])
            
    shapley_values[channel] = weighted_contribution
    
shapley_values

{'A': 14.999999999999996, 'B': 14.999999999999996, 'C': 20.0}

     Output should be A:15, B:15, C:20